In [22]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

2.10.0


In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [24]:
results = []
with open("../serial_collector/collected_data1.csv") as csvfile:
    reader = csv.reader(csvfile, quoting=csv.QUOTE_NONE) # change contents to floats
    for row in reader: # each row is a list
        results.append(row)

dataset = results
columns = ['action', 'accelX1', 'accelY1', 'accelZ1', 'gyroX1', 'gyroY1', 'gyroZ1',
           'accelX2', 'accelY2', 'accelZ2', 'gyroX2', 'gyroY2', 'gyroZ2',
           'accelX3', 'accelY3', 'accelZ3', 'gyroX3', 'gyroY3', 'gyroZ3',
           'accelX4', 'accelY4', 'accelZ4', 'gyroX4', 'gyroY4', 'gyroZ4',
           'accelX5', 'accelY5', 'accelZ5', 'gyroX5', 'gyroY5', 'gyroZ5']
data = pd.DataFrame(data = dataset, columns = columns)
data.head()

,action,accelX1,accelY1,accelZ1,gyroX1,gyroY1,gyroZ1,accelX2,accelY2,accelZ2,...,accelZ4,gyroX4,gyroY4,gyroZ4,accelX5,accelY5,accelZ5,gyroX5,gyroY5,gyroZ5
0,fel,2.926488,-0.441078,-1.713436,-73.192886,66.343773,188.981644,0,0,0,...,-5.138318,-35.824715,48.596447,40.759476,8.411429,6.196108,5.790317,28.070290,44.473473,11.749684
1,fel,0.216203,-1.292454,-2.366971,65.628464,65.616844,138.564056,0,0,0,...,0.167808,25.573467,11.763483,-45.573078,0,0,0,0,0,0
2,le,0.216203,-1.292454,-2.366971,65.628464,65.616844,138.564056,0,0,0,...,0.167808,25.573467,11.763483,-45.573078,0,0,0,0,0,0
3,fel,-0.958036,-1.440868,0.126096,65.600006,140.351822,-80.716515,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,fel,-1.316570,-0.630460,2.380210,-20.832790,57.284107,-26.551594,0,0,0,...,0,0,0,0,3.360665,3.440024,5.057559,-14.727086,-26.194059,5.754138


In [12]:
#data.shape => (x, y)
#data has x rows and y columns.
data.shape

(33, 31)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 31 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   action   33 non-null     object
 1   accelX1  33 non-null     object
 2   accelY1  33 non-null     object
 3   accelZ1  33 non-null     object
 4   gyroX1   33 non-null     object
 5   gyroY1   33 non-null     object
 6   gyroZ1   33 non-null     object
 7   accelX2  33 non-null     object
 8   accelY2  33 non-null     object
 9   accelZ2  33 non-null     object
 10  gyroX2   33 non-null     object
 11  gyroY2   33 non-null     object
 12  gyroZ2   33 non-null     object
 13  accelX3  33 non-null     object
 14  accelY3  33 non-null     object
 15  accelZ3  33 non-null     object
 16  gyroX3   33 non-null     object
 17  gyroY3   33 non-null     object
 18  gyroZ3   33 non-null     object
 19  accelX4  33 non-null     object
 20  accelY4  33 non-null     object
 21  accelZ4  33 non-null     object
 22  gyro

In [48]:
data.isnull().sum()

accelX1    0
accelY1    0
accelZ1    0
gyroX1     0
gyroY1     0
gyroZ1     0
accelX2    0
accelY2    0
accelZ2    0
gyroX2     0
gyroY2     0
gyroZ2     0
accelX3    0
accelY3    0
accelZ3    0
gyroX3     0
gyroY3     0
gyroZ3     0
accelX4    0
accelY4    0
accelZ4    0
gyroX4     0
gyroY4     0
gyroZ4     0
accelX5    0
accelY5    0
accelZ5    0
gyroX5     0
gyroY5     0
gyroZ5     0
dtype: int64

In [14]:
#To see the distribution of data we will see the count of each unique activity using value_counts().
data['action'].value_counts()

fel    33
Name: action, dtype: int64

In [25]:
label = LabelEncoder()
data['label'] = label.fit_transform(data['action'])
data.head()

,action,accelX1,accelY1,accelZ1,gyroX1,gyroY1,gyroZ1,accelX2,accelY2,accelZ2,...,gyroX4,gyroY4,gyroZ4,accelX5,accelY5,accelZ5,gyroX5,gyroY5,gyroZ5,label
0,fel,2.926488,-0.441078,-1.713436,-73.192886,66.343773,188.981644,0,0,0,...,-35.824715,48.596447,40.759476,8.411429,6.196108,5.790317,28.070290,44.473473,11.749684,0
1,fel,0.216203,-1.292454,-2.366971,65.628464,65.616844,138.564056,0,0,0,...,25.573467,11.763483,-45.573078,0,0,0,0,0,0,0
2,le,0.216203,-1.292454,-2.366971,65.628464,65.616844,138.564056,0,0,0,...,25.573467,11.763483,-45.573078,0,0,0,0,0,0,1
3,fel,-0.958036,-1.440868,0.126096,65.600006,140.351822,-80.716515,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,fel,-1.316570,-0.630460,2.380210,-20.832790,57.284107,-26.551594,0,0,0,...,0,0,0,3.360665,3.440024,5.057559,-14.727086,-26.194059,5.754138,0
